In [ ]:
import os
import openai
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import chroma, Pinecone
import pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv, find_dotenv
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from IPython.display import display, Markdown
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain


_ = load_dotenv(find_dotenv()) # read local .env file
#openai key
openai.api_key = os.environ['OPENAI_API_KEY']
openai.model_name = os.environ['FAST_LLM_MODEL']
print(openai.model_name)

In [ ]:
loader = PyPDFLoader("../data/8B3608368D6AA693562457E20559FE6C.pdf")

In [ ]:
data = loader.load()

In [139]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [140]:
embeddings = OpenAIEmbeddings()

In [ ]:
#init pinecone
pinecone.init(
    api_key=os.environ['PINECONE_API_KEY'],
    environment=os.environ['PINECONE_ENV'])
index_name = "pdftest"
pinecone.create_index("pdftest2", dimension=1536, metric="euclidean")
#pinecone.list_indexes()

In [ ]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [147]:
query = "这家公司的生产的设备有哪些？"
docs = docsearch.similarity_search(query)
#print(docs[0])
for t in docs:
    print(t)

page_content='1-1-25 搭建公司产品架 构，并向其关键零部件进口替代进行应用物理的深入研发。与\n此同时，充分发挥自身的竞争优势，积极探索新技术、新产品，把握行业发展\n趋势，不断进行技术创新和产品革新，为客户提供更加符合其发展目标、多元\n化的智能制造解决方案，立足于自主研发、掌握核心科技，力争成为全球领先\n的智能制造高端装备供应商。  \n1、锂离子电池生产设备及其关键零部件领域  \n在锂电池干燥设备领域，公司将继续保持竞争优势，不断进行技术革新，\n未来三年内，公司将开发智能式冷热液流高真空烤箱和智能高效小单元式烘烤\n线并向市场推广，实现该领域的全自动完全无人化，将大幅 提升烘烤效率、降\n低运营成本。  \n在锂电池涂布设备领域，公司目前已采用了先进的高精度单向双面挤压涂\n布技术，大大提升了涂布效率及均匀性。在未来，公司将继续发挥该领域的技\n术优势，开发符合行业发展趋势的微孔基材双面挤压涂布机及电极干法涂布机。  \n在锂电池辊压、分切设备领域，公司已开发出全自动高速辊压分切一体设\n备，未来将继续推进该领域自动化程度、生产效率的不断提升。  \n在锂电设备关键零部件领域，公司利用精密高速涂布模头技术开发的高精\n密挤压模头已具备行业领先水平，未来将大力推进该领域的产业化，实现该领\n域的进口替代。  \n2、其他自动化设备领域  \n在其他自动化设备领域，公司将持续开拓光电设备、医疗用品设备、光学\n膜、氢燃料电池市场，不断在新的领域提升核心竞争力及市场占有率，拓展公\n司业绩新的增长点。  \n以上具体情况参见本招股说明书 “第七节 募集资金运用与未来发展规划 ”。 \n十一、其他对发行人有重大影响的事项  \n截至本招股说明书签署日，发行人不存在其他有重大影响的事项。' metadata={}
page_content='1-1-25 搭建公司产品架 构，并向其关键零部件进口替代进行应用物理的深入研发。与\n此同时，充分发挥自身的竞争优势，积极探索新技术、新产品，把握行业发展\n趋势，不断进行技术创新和产品革新，为客户提供更加符合其发展目标、多元\n化的智能制造解决方案，立足于自主研发、掌握核心科技，力争成为全球领先\n的智能制造高端装备供应商。  \n1、锂离子电池生产设备及其关键零部件领域  \n在锂电池干燥设备领域，公

In [ ]:
template = ChatPromptTemplate.from_template(
    "This is a listing prospectus, suppose you a Stock investor,\
    Base on the information the information below:\
    {docs}, you need to find out the detial information \
     to answer the quetion of {query}\
     then output the anser in Chinese. NOte that, Do not summary \
     the information you found , but list the detail of them. \
     finally, output the answer in Chinese."
)
print(template)

In [ ]:
llm = OpenAI(model_name="gpt-3.5-turbo-0613",temperature=0.0)
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
response = chain.run(input_documents=docs, question=query, template = template)
display(Markdown(response))